In [1]:
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
data = pd.read_csv("row_data\건강보험심사평가원_요양기관 개설 현황_20241231.csv", encoding="cp949")

data.head()

,암호화된요양기호,요양기관명,요양종별,시도명,시군구명,도로명주소,표시과목명,개설일자
0,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkMzgxOTYxIzQxIyQxIy...,"""사""대한산업보건협회부설경북의원",의원,경상북도,경산시,경상북도 경산시 진량읍 공단1로 28,일반의,1995-01-24
1,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkNDgxOTYxIzIxIyQxIy...,(VOM)봄안과의원,의원,대구광역시,동구,"대구광역시 동구 안심로 58, 3층 (율하동, 율하메디빌)",안과,2014-01-02
2,JDQ4MTAxMiM1MSMkMSMkMCMkOTkkNTgxOTYxIzExIyQxIy...,(복지)예사랑병원,정신병원,부산광역시,서구,부산광역시 서구 옥천로130번길 38 (아미동2가),해당사항없음,2012-02-29
3,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkNDgxOTYxIzUxIyQxIy...,(사) 경찰공제회 포항의원,의원,경상북도,포항시 남구,경상북도 포항시 남구 오천읍 냉천로 656,일반의,2001-06-13
4,JDU4MTI3MSM1MSMkMiMkMCMkMDAkNDgxOTYxIzIxIyQxIy...,(사)경찰공제회남부의원,의원,인천광역시,남동구,인천광역시 남동구 아암대로 1247 (고잔동),일반의,1998-09-28


In [3]:
temp = data[["요양기관명", "시도명", "개설일자"]]
temp["개설일자"]=pd.to_datetime(temp["개설일자"], errors="coerce")
temp["개설연도"]=temp["개설일자"].dt.year
print(temp.head())

temp['연도그룹'] = temp['개설연도'].apply(
    lambda x: '2008이전' if x <= 2008 else str(x)
)
grouped = temp.groupby(['시도명', '연도그룹']).size().reset_index(name='신규기관수')
order = ['2008이전'] + [str(y) for y in range(2009, 2025)]
grouped['연도그룹'] = pd.Categorical(grouped['연도그룹'], categories=order, ordered=True)
grouped = grouped.sort_values(['시도명', '연도그룹'])
grouped['누적기관수'] = grouped.groupby('시도명')['신규기관수'].cumsum()

cnt_groupby_region_year=grouped
print(cnt_groupby_region_year)

               요양기관명    시도명       개설일자  개설연도
0  "사"대한산업보건협회부설경북의원   경상북도 1995-01-24  1995
1         (VOM)봄안과의원  대구광역시 2014-01-02  2014
2          (복지)예사랑병원  부산광역시 2012-02-29  2012
3     (사) 경찰공제회 포항의원   경상북도 2001-06-13  2001
4       (사)경찰공제회남부의원  인천광역시 1998-09-28  1998
         시도명    연도그룹  신규기관수  누적기관수
0    강원특별자치도  2008이전   1443   1443
1    강원특별자치도    2009     61   1504
2    강원특별자치도    2010     58   1562
3    강원특별자치도    2011     65   1627
4    강원특별자치도    2012     71   1698
..       ...     ...    ...    ...
284     충청북도    2020     88   2593
285     충청북도    2021    106   2699
286     충청북도    2022     75   2774
287     충청북도    2023     82   2856
288     충청북도    2024     70   2926

[289 rows x 4 columns]


C:\Users\wjdwn\AppData\Local\Temp\ipykernel_41452\1519383199.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["개설일자"]=pd.to_datetime(temp["개설일자"], errors="coerce")
C:\Users\wjdwn\AppData\Local\Temp\ipykernel_41452\1519383199.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["개설연도"]=temp["개설일자"].dt.year
C:\Users\wjdwn\AppData\Local\Temp\ipykernel_41452\1519383199.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [11]:
cnt_groupby_region_year.to_csv("pre_processing_data/cnt_groupby_region_year.csv", index=False, encoding="utf-8-sig")

In [14]:
def process_population_data(df_input, year):
    df = df_input[[
        "행정구역",
        f"{year}년_계_총인구수",
        f"{year}년_계_0~9세",
        f"{year}년_계_60~69세",
        f"{year}년_계_70~79세",
        f"{year}년_계_80~89세",
        f"{year}년_계_90~99세",
        f"{year}년_계_100세 이상"
    ]].copy()

    original_columns = [
        '행정구역', '총인구수', '0~9세', '60~69세', '70~79세', '80~89세', '90~99세', '100세이상'
    ]
    df.columns = original_columns

    numeric_cols = original_columns[1:]

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '', regex=False), errors='coerce')

    age_columns_to_sum = ['60~69세', '70~79세', '80~89세', '90~99세', '100세이상']
    df['노인인구수'] = df[age_columns_to_sum].sum(axis=1)

    df['년도'] = year


    df['행정구역'] = df['행정구역'].replace({'강원도 (4200000000)': '강원특별자치도 (4200000000)','전라북도 (4500000000)': '전북특별자치도 (4500000000)'})
    df['행정구역'] = df['행정구역'].str.replace(r'\s+\(.+\)', '', regex=True)

    final_df = df.rename(columns={'0~9세': '소아인구수'})
    return final_df[['행정구역', '년도', '총인구수', '소아인구수', '노인인구수']]

In [15]:
start_year = 2008
end_year = 2024
years = range(start_year, end_year + 1)
all_data_list = []
base_path = "row_data/"

for year in years:
    file_name = f"{year}12_{year}12_연령별인구현황_연간.csv"
    file_path = base_path + file_name

    data_frame = pd.read_csv(r"{}".format(file_path), encoding='cp949') 
    
    processed_df = process_population_data(data_frame, year)
    
    all_data_list.append(processed_df)

final_combined_df = pd.concat(all_data_list, ignore_index=True)
        

In [17]:
final_combined_df.to_csv("pre_processing_data\population_groupby_region_year.csv", index=False, encoding="utf-8-sig")

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\wjdwn\AppData\Local\Temp\ipykernel_41452\328933788.py:1: SyntaxWarning: invalid escape sequence '\p'
  final_combined_df.to_csv("pre_processing_data\population_groupby_region_year.csv", index=False, encoding="utf-8-sig")


In [21]:
cnt_groupby_region_year = pd.read_csv("pre_processing_data\cnt_groupby_region_year.csv")
population_groupby_region_year = pd.read_csv("pre_processing_data/population_groupby_region_year.csv")
print(cnt_groupby_region_year)
print()
print(population_groupby_region_year)

         시도명    연도그룹  신규기관수  누적기관수
0    강원특별자치도  2008이전   1443   1443
1    강원특별자치도    2009     61   1504
2    강원특별자치도    2010     58   1562
3    강원특별자치도    2011     65   1627
4    강원특별자치도    2012     71   1698
..       ...     ...    ...    ...
284     충청북도    2020     88   2593
285     충청북도    2021    106   2699
286     충청북도    2022     75   2774
287     충청북도    2023     82   2856
288     충청북도    2024     70   2926

[289 rows x 4 columns]

        행정구역    년도      총인구수    소아인구수    노인인구수
0         전국  2008  49540367  5096863  7110229
1      서울특별시  2008  10200827   930515  1342702
2      부산광역시  2008   3564577   295585   538619
3      대구광역시  2008   2492724   243689   331818
4      인천광역시  2008   2692696   282010   311800
..       ...   ...       ...      ...      ...
297  전북특별자치도  2024   1738690    95081   591893
298     전라남도  2024   1788819   101368   651577
299     경상북도  2024   2531384   140258   898500
300     경상남도  2024   3228380   195007   993526
301  제주특별자치도  2024    670368    47756 

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\wjdwn\AppData\Local\Temp\ipykernel_36596\3624163032.py:1: SyntaxWarning: invalid escape sequence '\c'
  cnt_groupby_region_year = pd.read_csv("pre_processing_data\cnt_groupby_region_year.csv")


In [22]:
population_groupby_region_year['행정구역'] = (
    population_groupby_region_year['행정구역']
    .replace({
        "강원도": "강원특별자치도",
        "전라북도": "전북특별자치도"
    })
)

In [23]:
cnt_groupby_region_year['년도'] = cnt_groupby_region_year['연도그룹'].replace({'2008이전': '2008'}).astype(int)

df_merged = pd.merge(
    cnt_groupby_region_year,
    population_groupby_region_year.rename(columns={'행정구역':'시도명', '년도':'년도'}),
    on=['시도명', '년도'],
    how='left'
)

df_merged['노인인구_기관수_비율'] = df_merged['누적기관수'] / df_merged['노인인구수']
print(df_merged)

         시도명    연도그룹  신규기관수  누적기관수    년도       총인구수     소아인구수     노인인구수  \
0    강원특별자치도  2008이전   1443   1443  2008  1508575.0  148237.0  282916.0   
1    강원특별자치도    2009     61   1504  2009  1512870.0  142407.0  289564.0   
2    강원특별자치도    2010     58   1562  2010  1529818.0  136122.0  299088.0   
3    강원특별자치도    2011     65   1627  2011  1536448.0  132030.0  304331.0   
4    강원특별자치도    2012     71   1698  2012  1538630.0  129660.0  318860.0   
..       ...     ...    ...    ...   ...        ...       ...       ...   
284     충청북도    2020     88   2593  2020  1600837.0  123371.0  419207.0   
285     충청북도    2021    106   2699  2021  1597427.0  116975.0  440818.0   
286     충청북도    2022     75   2774  2022  1595058.0  109410.0  459521.0   
287     충청북도    2023     82   2856  2023  1593469.0  103333.0  473101.0   
288     충청북도    2024     70   2926  2024  1591177.0   97837.0  488859.0   

     노인인구_기관수_비율  
0       0.005100  
1       0.005194  
2       0.005223  
3       0.005346  
4   

In [24]:
print(df_merged.isna().sum())

시도명            0
연도그룹           0
신규기관수          0
누적기관수          0
년도             0
총인구수           4
소아인구수          4
노인인구수          4
노인인구_기관수_비율    4
dtype: int64


In [44]:
df_merged = df_merged.sort_values(['시도명','년도'])

df_merged.loc[(df_merged['시도명'] == '세종특별자치시') & (df_merged['년도'] < 2012), '년도'] = pd.NA

df_clean = df_merged.dropna(subset=['년도'])

df_clean['년도'] = df_clean['년도'].astype(int)

rate_groupby_region_year = df_clean[["시도명", "년도", "노인인구_기관수_비율"]]

C:\Users\wjdwn\AppData\Local\Temp\ipykernel_36596\2610259844.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['년도'] = df_clean['년도'].astype(int)


In [45]:
rate_groupby_region_year.to_csv("pre_processing_data/rate_groupby_region_year.csv", index=False, encoding="utf-8-sig")


In [46]:
cnt_groupby_region_year.head()


,시도명,연도그룹,신규기관수,누적기관수,년도
0,강원특별자치도,2008이전,1443,1443,2008
1,강원특별자치도,2009,61,1504,2009
2,강원특별자치도,2010,58,1562,2010
3,강원특별자치도,2011,65,1627,2011
4,강원특별자치도,2012,71,1698,2012


In [47]:
population_groupby_region_year.head()


,행정구역,년도,총인구수,소아인구수,노인인구수
0,전국,2008,49540367,5096863,7110229
1,서울특별시,2008,10200827,930515,1342702
2,부산광역시,2008,3564577,295585,538619
3,대구광역시,2008,2492724,243689,331818
4,인천광역시,2008,2692696,282010,311800


In [48]:
rate_groupby_region_year.head()

,시도명,년도,노인인구_기관수_비율
0,강원특별자치도,2008,0.005100
1,강원특별자치도,2009,0.005194
2,강원특별자치도,2010,0.005223
3,강원특별자치도,2011,0.005346
4,강원특별자치도,2012,0.005325
